In [1]:
%%capture
! pip install investpy

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
import seaborn as sns

In [4]:
import logging
import os
import sys

In [5]:
import investpy

In [6]:
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# Set working directory:
PATH_MAIN = os.path.join(ROOT, 'My Drive/DS/Extra Year/BDOS/scripts') 
os.chdir(PATH_MAIN)
print('The Current Directory is: {}'.format( os.getcwd() ) )

Mounted at /content/drive
The Current Directory is: /content/drive/My Drive/DS/Extra Year/BDOS/scripts


In [7]:
# Custom Libraries:
from importlib.machinery import SourceFileLoader
path_retriever = SourceFileLoader('path_retriever', os.path.join(PATH_MAIN, 'lib/path_retriever.py')).load_module()
preprocessing = SourceFileLoader('preprocessing', os.path.join(PATH_MAIN, 'lib/preprocessing.py')).load_module()


In [8]:
# Initialize the logger:
import logging.config
import yaml
PATH_CONFIGS = path_retriever.get_path('configs')
with open(os.path.join(PATH_CONFIGS, 'log_configs.yml'), 'rt') as f:
  config = yaml.safe_load(f.read())
  logging.config.dictConfig(config)

# clean the log file if exists:
PATH_LOG = path_retriever.get_path('log')
LOG_FILE = os.path.join(PATH_LOG, 'my_logging.log')
if os.path.isfile(LOG_FILE):
  os.remove(LOG_FILE)

# Get Data


In [137]:
START = '01/01/2019'
END = '31/12/2021'

### 1. Preprocessing (Data Retrieval + Transformations):

In [ ]:
target_df, crypto_df = preprocessing.retrieve_data(start_period=START, end_period=END)

# Stilized Facts: